# Object Detection Video Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import cv2
#import pyttsx3;
#engine = pyttsx3.init();
#voices = engine.getProperty('voices')



cap = cv2.VideoCapture(0)

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
sys.path

['c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\python36.zip',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\DLLs',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36',
 '',
 'C:\\Users\\DA1030TU\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\object_detection-0.1-py3.6.egg',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\win32',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\win32\\lib',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\Pythonwin',
 'c:\\users\\da1030tu\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'trained'


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('trained', 'label_map.pbtxt')

NUM_CLASSES = 3

## Download Model

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

# Detection

In [8]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            min_score_thresh = 0.5
           
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(25) & 0xFF == ord('q'): 
                cv2.destroyAllWindows()
                break


# After the loop release the cap object 
cap.release() 
# Destroy all the windows 
cv2.destroyAllWindows()     